In [1]:
import pandas as pd
import pdfplumber
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
from openpyxl import load_workbook
from datetime import datetime


pdf_path = "/home/jellyfish/Simalarity/Grey, Meredith SAP 11.16.23.pdf"
def ocr_pdf(pdf_path):
    all_text = ''
    with fitz.open(pdf_path) as doc:
        for page in doc:
            image_list = page.get_images(full=True)
            for image_index, img in enumerate(page.get_images(full=True)):
                # Extract the image bytes
                base_image = doc.extract_image(img[0])
                image_bytes = base_image["image"]

                # Convert the bytes to a PIL image
                image = Image.open(io.BytesIO(image_bytes))

                # Perform OCR using pytesseract
                text = pytesseract.image_to_string(image, lang='eng')

                all_text += text + '\n'
    return all_text

# Extract text from the PDF via OCR
sap_text = ocr_pdf(pdf_path)



In [2]:
### LP
lp_text = ''
with pdfplumber.open('/home/jellyfish/Simalarity/Grey, Meredith LP 11.16.23 (1).pdf') as pdf:
    for i in range(len(pdf.pages)):
        lp_text = lp_text + '\n' +pdf.pages[i].extract_text()
lp_text = lp_text.strip()
print(lp_text)
    

Name :Meredith Grey Effective From : 11/16/2023 Version : 4.00
TABS ID: 654321 Life Plan Type : Annual Status:Finalized
Life Plan
Member Information
Name : Meredith Grey Date of Birth : 11/10/1996
Address : 456 Wallaby Way Medford, NY 11763 Phone : 631-987-6543
Medicaid : CD56789E Medicare :
Willowbrook Member : No Enrollment Date : 07/01/2018
CCO Information
Care Manager Name : Christina Yang Care Manager Email : Christina.Yang@myacany.org
Care Manager Phone : 347-888-8888 Care Manager Advance Care Alliance, 300 Motor
Address :
Parkway, Suite 105, Hauppauge, NY,
Care Manager Fax : 646-777-7777 11788
Supervisor Name : Miranda Bailey Supervisor Email : Miranda.Bailey@myacany.org
Supervisor Phone : 929-666-6666
CCO Name : Advance Care Alliance of NY CCO Address : 300 Motor Parkway Hauppauge NY
11788
Phone : ___-___-____
Emergency/After Hours 833-456-7890
Fax : ___-___-____ Phone :
Printed on 12/21/2023 11:09 AM Page 1 of 14
Name :Meredith Grey Effective From : 11/16/2023 Version : 4.00
T

In [3]:
from langchain_community.llms.ollama import Ollama
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_text_splitters.spacy import SpacyTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders.web_base import WebBaseLoader

embeddings = HuggingFaceBgeEmbeddings()
llm = Ollama(model='llama2',temperature=0.8)
textSplitter = SpacyTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = textSplitter.split_text(lp_text)
db  = FAISS.from_texts(documents,embeddings)

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based on only provided context:
    <context>
    {context}
    </context>

    Question: {input}
"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)




/home/jellyfish/Simalarity/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jellyfish/Simalarity/env/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [4]:
response = retrieval_chain.invoke({"input": "Give me name,phone no. , address, insight in key value pair"})

print(response["answer"])

Sure, based on the provided context, here are the name, phone number, address, and insights for each person or organization listed:

1. Grey, Ellis - Mother
	* Name: Grey, Ellis
	* Phone Number: 631-987-6543 (mentioned in the context)
	* Address: 456 Wallaby Way, Medford, NY 11763 (mentioned in the context)
	* Insight: Ellis Grey is Meredith's mother and a key contact for her life plan.
2. Ollerenshaw, Gloria - Support Broker
	* Name: Ollerenshaw, Gloria
	* Phone Number: Not mentioned in the context
	* Address: Not mentioned in the context
	* Insight: Gloria Ollerenshaw is Meredith's support broker and played a role in facilitating Meredith's life plan review.
3. Camello, Jennifer - Fiscal Intermediary
	* Name: Camello, Jennifer
	* Phone Number: Not mentioned in the context
	* Address: Not mentioned in the context
	* Insight: Jennifer Camello is Meredith's fiscal intermediary and was involved in the life plan review process.
4. Yang, Christina - Care Manager
	* Name: Yang, Christina
	*

In [6]:
response = retrieval_chain.invoke({"input": "Extract patient name, address, contact no. ,Tabs id,birth date , care Manager name, Care Manager address, Care Manager phone no., RCIL in form of key value pair"})

print(response["answer"])

Sure, I can help you with that! Based on the provided context, here are the patient's details that can be extracted:

Patient Name: Meredith Grey
Address: 456 Wallaby Way, Medford, NY 11763
Contact Number: 631-987-6543
TABS ID: 654321
Birth Date: November 10, 1996
Care Manager Name: Christina Yang
Care Manager Address: Care Manager Email: Christina.Yang@myacany.org
Care Manager Phone Number: 347-888-8888
RCIL (Resource Community): Advance Care Alliance of NY
RCIL Address: 300 Motor Parkway, Hauppauge, NY 11788
RCIL Phone Number: ___-___-____

Note that some of the information is not explicitly provided in the context, such as the patient's middle name or any additional contact details.
